In [4]:
import os
import cv2
import pickle
import numpy as np
import face_recognition
import cvzone
from gaze_tracking import GazeTracking

background = cv2.imread('resources/background.png')
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# Importing the mode images into a list
folderModePath = 'C:/Users/Sameer/OneDrive/Desktop/Everything/python/curr/Resources/modes/'
modesPathList = os.listdir(folderModePath)
imgModeList = []

for path in modesPathList:
    imgModeList.append(cv2.imread(os.path.join(folderModePath, path)))

# load the encoded file
print("Loading Encode File ...")
file = open('EncodeFile.p', 'rb')
encodeListKnownWithIds = pickle.load(file)
file.close()
encodeListKnown, studentIds = encodeListKnownWithIds
print("Encode File Loaded!")

# Initialize gaze tracking
gaze = GazeTracking()

modeType = 0
counter = 0
id = -1
imgStudent = []

while True:
    # Capture frame-by-frame
    success, img = cap.read()

    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    faceCurFrame = face_recognition.face_locations(imgS)
    encodeCurFrame = face_recognition.face_encodings(imgS, faceCurFrame)

    background[162:162 + 480, 55:55 + 640] = img
    background[44:44 + 633, 808:808 + 414] = imgModeList[modeType]

    flag = 0
    if faceCurFrame:
        for encodeFace, faceLoc in zip(encodeCurFrame, faceCurFrame):
            matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
            faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)

            matchIndex = np.argmin(faceDis)
            if matches[matchIndex]:
                y1, x2, y2, x1 = faceLoc
                y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
                bbox = 55 + x1, 162 + y1, x2 - x1, y2 - y1
                background = cvzone.cornerRect(background, bbox, rt=0)
                id = studentIds[matchIndex]
                print('Face matched')
                flag = 1
            else:
                print("Match not found!")

    # Perform gaze tracking
    gaze.refresh(imgS)
    left_gaze = gaze.is_left()
    right_gaze = gaze.is_right()

    if left_gaze:
        print("Gaze: Left")
    elif right_gaze:
        print("Gaze: Right")
    else:
        print("Gaze: Center")

    if flag:
        break

    # Display the frame
    cv2.imshow("Attendance Sys", background)

    # Press 'q' to exit the loop and close the camera
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close the OpenCV window
cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'gaze_tracking'